In [2]:
import pandas as pd

ds = pd.read_json("../medical_data/Corona2.json")
ds.head(5)

,examples
0,"{'id': '18c2f619-f102-452f-ab81-d26f7e283ffe',..."
1,"{'id': '487c93e3-0d45-4088-a378-cf3a01c8953d',..."
2,"{'id': 'd5056874-895a-4a7f-9e0f-828d414d65d9',..."
3,"{'id': '20c792c7-0c4b-42d0-8127-0e04113db384',..."
4,"{'id': 'f5359e0d-4d4a-4707-95a3-4c627fc4a83b',..."


In [14]:
import json
with open("../medical_data/Corona2.json", "r") as f:
    data = json.load(f)

examples = data["examples"]
df_main = pd.DataFrame(examples)
display(df_main.head())

,id,content,metadata,annotations,classifications
0,18c2f619-f102-452f-ab81-d26f7e283ffe,While bismuth compounds (Pepto-Bismol) decreas...,{},[{'id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed'...,[]
1,487c93e3-0d45-4088-a378-cf3a01c8953d,"Diarrhea, also spelled diarrhoea, is the condi...",{},[{'id': '28601a42-c8a9-44e2-aeea-8939cb1db1a9'...,[]
2,d5056874-895a-4a7f-9e0f-828d414d65d9,Antiretroviral therapy (ART) is recommended fo...,{},[{'id': '0a4b4cc2-3a07-4817-8ccc-8fa1f945573d'...,[]
3,20c792c7-0c4b-42d0-8127-0e04113db384,The following drugs are considered as DMARDs: ...,{},[{'id': '0e370ff5-fee6-45e1-adcb-d5f00fa33b0d'...,[]
4,f5359e0d-4d4a-4707-95a3-4c627fc4a83b,"The goals of treatment are to reduce pain, dec...",{},[{'id': '10411ff5-e343-40bf-95d5-39cd1cb16060'...,[]


# TODO:
* read https://mageswaran1989.medium.com/how-to-build-custom-ner-huggingface-dataset-for-receipts-and-train-with-huggingface-transformers-6c954b84473c
* get all tags
* get tokens